In [76]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
from bokeh.resources import INLINE
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [77]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display
# Configure the map plot
humidity_in_city = city_data_df.hvplot.points("Lng",
                                             "Lat",
                                             geo = True,
                                             width = 800,
                                             height = 400,
                                             color = "City",
                                             size = "Humidity",
                                             tiles = "OSM")

# Display the map
humidity_in_city

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] <= 23) & (city_data_df["Max Temp"] >= 22) & (city_data_df["Humidity"] >= 10) & (city_data_df["Cloudiness"] >= 10) & (city_data_df["Wind Speed"] <= 14)]
# Drop any rows with null values
# N/A
# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
19,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
27,27,bambous virieux,-20.3428,57.7575,22.14,68,75,3.60,MU,1666108246
79,79,mahebourg,-20.4081,57.7000,22.16,68,75,3.60,MU,1666108278
87,87,saint-philippe,-21.3585,55.7679,22.77,76,37,4.08,RE,1666108284
142,142,peniche,39.3558,-9.3811,22.82,89,84,8.69,PT,1666108321
189,189,grand gaube,-20.0064,57.6608,22.22,65,47,0.89,MU,1666108353
218,218,sao teotonio,37.5128,-8.7071,22.45,60,31,4.98,PT,1666108367
229,229,san cristobal,7.7669,-72.2250,22.10,45,46,0.57,VE,1666108372


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so yan store theou c hotel found using the Geoapify API
hotel_df.insert(5,"Hotel", "" ) #coloumn, new column name, list

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel
0,faya,SA,18.3851,42.4509,35,
19,puerto ayora,EC,-0.7393,-90.3518,82,
20,avarua,CK,-21.2078,-159.7750,73,
27,bambous virieux,MU,-20.3428,57.7575,68,
79,mahebourg,MU,-20.4081,57.7000,68,
87,saint-philippe,RE,-21.3585,55.7679,76,
142,peniche,PT,39.3558,-9.3811,89,
189,grand gaube,MU,-20.0064,57.6608,65,
218,sao teotonio,PT,37.5128,-8.7071,60,
229,san cristobal,VE,7.7669,-72.2250,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"
params = {
    "apiKey": geoapify_key,
    "limit": limit,
    "categories": categories,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row ["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
bambous virieux - nearest hotel: Otentic Eco Tent Experience
mahebourg - nearest hotel: Grand Bel Air
saint-philippe - nearest hotel: Le Baril
peniche - nearest hotel: Residencial Maciel
grand gaube - nearest hotel: Hotel Lux- Grand Gaube
sao teotonio - nearest hotel: Orada Velha
san cristobal - nearest hotel: Terra Sur
quatre cocos - nearest hotel: Horisun
camacha - nearest hotel: Praia Dourada
jalu - nearest hotel: فندق جالو السياحي
souillac - nearest hotel: Hotel SkylineInn
viseu - nearest hotel: No hotel found
saldanha - nearest hotel: Hoedjesbaai
salto - nearest hotel: Hotel Español Salto
chivilcoy - nearest hotel: Falcone
ilhabela - nearest hotel: Vila Kebaya


,City,Country,Lat,Lng,Humidity,Hotel,Hotel Name
0,faya,SA,18.3851,42.4509,35,,No hotel found
19,puerto ayora,EC,-0.7393,-90.3518,82,,Hostal La Mirada De Solitario George
20,avarua,CK,-21.2078,-159.7750,73,,Paradise Inn
27,bambous virieux,MU,-20.3428,57.7575,68,,Otentic Eco Tent Experience
79,mahebourg,MU,-20.4081,57.7000,68,,Grand Bel Air
87,saint-philippe,RE,-21.3585,55.7679,76,,Le Baril
142,peniche,PT,39.3558,-9.3811,89,,Residencial Maciel
189,grand gaube,MU,-20.0064,57.6608,65,,Hotel Lux- Grand Gaube
218,sao teotonio,PT,37.5128,-8.7071,60,,Orada Velha
229,san cristobal,VE,7.7669,-72.2250,45,,Terra Sur


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [80]:
%%capture --no-display
# Configure the map plot
hotel_op = hotel_df.hvplot.points("Lng", "Lat",geo = True,width = 800,height = 400,color = "City", size = "Humidity",tiles = "OSM", hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_op
# hvplot.save(hotel_op,"hotel_op_fig.html", resources=INLINE)
# hvplot.save(hotel_op, "hotel_op_fig.png")
# Path("hotel_op_fig.html").unlink(missing_ok=True)
# Path('hotel_op_fig.png').unlink(missing_ok=True)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)